# Getting Started

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import tensorflow as tf

In [ ]:
tf.random.set_seed(4445444)

In [ ]:
train_df = pd.read_csv('../input/human-activity-recognition-with-smartphones/train.csv')
test_df = pd.read_csv('../input/human-activity-recognition-with-smartphones/test.csv')

In [ ]:
train_df.info()

In [ ]:
print("Train set missing values:", train_df.isna().sum().sum())

In [ ]:
test_df.info()

In [ ]:
print("Test set missing values:", test_df.isna().sum().sum())

# Splitting Data

In [ ]:
train_df

In [ ]:
y_train = train_df['Activity'].copy()
X_train = train_df.drop('Activity', axis=1).copy()

y_test = test_df['Activity'].copy()
X_test = test_df.drop('Activity', axis=1).copy()

# Encoding Labels

In [ ]:
y_train.value_counts()

In [ ]:
num_classes = 6

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

# Scaling Data

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Modeling/Training

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
batch_size = 32
epochs = 25

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('./model.h5', save_best_only=True),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,
            patience=1
        )
    ]
)

# Results

In [ ]:
plt.figure(figsize=(16, 10))

epochs_range = range(epochs)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(
    epochs_range,
    train_loss,
    label="Training Loss"
)

plt.plot(
    epochs_range,
    val_loss,
    label="Validation Loss"
)

plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.legend()
plt.title("Loss Over Time")

plt.show()

In [ ]:
model.load_weights('./model.h5')

In [ ]:
model.evaluate(X_test, y_test)